In [1]:
notes = [
    ' 2_LINE  5_LINE Uptown trains run express from 3 Av-149 St to E 180 St',
    ' 2_LINE Downtown trains run local from 96 St to Times Sq-42 St',
    ' 2_LINE Flatbush Av-bound trains skip Nereid Av, 233 St, 225 St and 219 St',
    ' 2_LINE trains replace weekend  5_LINE service between Dyre Av and E 180 St',
    ' 4_LINE All trains from Junius St to Sutter Av-Rutland Rd board at the New Lots Av-bound platform',
    ' 4_LINE No trains between Utica Av and New Lots Av',
    ' A_LINE Trains make local stops in both directions at 155 St and 163 St',
    ' D_LINE  N_LINE  R_LINE Prospect Av is closed for renovation',
    ' F_LINE Jamaica-bound trains are rerouted via the  E_LINE after 47-50 Sts to Roosevelt Av',
    ' F_LINE Manhattan-bound platforms at Avenue U, Avenue P, Avenue N, Bay Pkwy  and Avenue I are closed for renovation',
    ' L_LINE Service operates every 12 minutes between Myrtle-Wyckoff Avs and Rockaway Pkwy',
    ' M_LINE Service ends early between 71 Av and Essex St E_LINE  F_LINE  R_LINE trains provide alternate service',
    ' R_LINE Service restored at Bay Ridge Av',
    ' sir_LINE Grant City Station Entrance Closure'
    

]

In [2]:
note_type= [
    'run express',
    'run local',
    'skip',
    'replace',
    'platform change',
    'no train',
    'make stop',
    'close',
    'reroute',
    'platform close',
    'operation frequency',
    'end_early',
    'restore',
    'entrance closure'
    
]

In [3]:
import pandas as pd
cat = pd.DataFrame({'notes': notes, 'type': note_type})
#cat.to_csv('notes&type.csv', index = False)

In [100]:
import re
def parse(a):
    dic = None
    
    ## rerouted
    if 'rerouted' in a:

        a_split = a.split('rerouted')
        first = a_split[0]
        second = a_split[1]

        ##find lines reroute other
        line_a = re.findall('[a-zA-Z0-9]*_LINE', second)

        ##find lines that will be rerouted
        line_b = re.findall('[a-zA-Z0-9]*_LINE', first)

        #direction
        bound = re.findall('_LINE (.*?) trains', first)

        ##between
        stations = re.findall('after (.*?) to (.*)', a)
        start = stations[0][0]
        end = stations[0][1]


        dic = {'type': 'rerout',
               'rerouted by': line_a,
               'rerouting': line_b,
               'from': start,
               'to': end }
    

    elif 'platforms' in a and 'closed' in a:
        #type
        typee = 'platforms closed'

        linebound, rest = a.split('platforms')

        #line
        line = re.findall('[a-zA-Z0-9]*_LINE', linebound)

        #direction
        bound = re.findall('[a-zA-Z0-9]*-bound', linebound)

        #reason
        reason = re.findall('for (.*)', rest)

        #stations
        stations_s = re.findall('platforms at (.*?) closed ', a)[0].strip(' are').strip(' is')
        stations = re.split(', | and ',stations_s)

        dic = {'type': typee,
              'direction': bound[0],
              'reason': reason[0],
              'stations':stat,
              'lines': line}  
    
    
    ## close
    elif 'closed' in a:

        ## find lines 
        lines = re.findall('[a-zA-Z0-9]*_LINE', a)

        ## stations
        last_station = lines[-1]
        s = last_station + ' (.*?) is'
        stations_s = re.findall(s, a)[0]
        stations = re.split(', | and ',stations_s)


        ## reason 
        reason = re.findall('closed for (.*)', a)[0]

        dic = {'type': 'change platform',
               'lines': lines,
               'stations': stations,
               'reason': reason}
    
    ## local stops
    elif 'local stops' in a:

        ## find lines 
        lines = re.findall('[a-zA-Z0-9]*_LINE', a)

        ## direction
        bound = re.findall('in (.*?) directions', a)[0]

        ## stations
        stations_s = re.findall('local stops (.*?) at (.*)', a)[0][1]
        stations = re.split(', | and ',stations_s)

        dic = {'type': 'local stops',
               'lines': lines,
               'direction': bound ,
               'stations': stations}
    
    
    
    ## platform change
    elif 'platform' in a and 'board' in a:

        ##find lines 
        lines = re.findall('[a-zA-Z0-9]*_LINE', a)

        ##from to
        stations = re.findall('from (.*?) to (.*) board', a)
        start = stations[0][0]
        end = stations[0][1]

        ##new platform
        platform = re.findall('at the (.*?) platform', a)[0]

        
        dic = {'type': 'change platform',
               'lines': lines,
               'board at': platform,
               'from': start,
               'to': end }
        
        
    ## replace
    elif 'replace' in a:

        a_split = a.split('replace')
        first = a_split[0]
        second = a_split[1]

        ##find lines replacing other
        line_a = re.findall('[a-zA-Z0-9]*_LINE', first)

        ##find lines that will be replaced
        line_b = re.findall('[a-zA-Z0-9]*_LINE', second)

        ##find time 
        time = re.findall('replace (.*?)  [a-zA-Z0-9]*_LINE', a)[-1]

        ##between
        stations = re.findall('between (.*?) and (.*)', a)
        start = stations[0][0]
        end = stations[0][1]


        dic = {'type': 'replace',
               'replaced by': line_a,
               'replacing': line_b,
               'time': time,
               'from': start,
               'to': end }
        
    
    ## skip
    elif 'skip' in a:
        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE', a)
        
        ##find bound
        last_line = line[-1]
        text = str(last_line) + r' (.*?) trains'
        bound = re.findall(text, a)[0]
        
        ##Stations
        stations_s = re.findall('skip (.*)', a)
        stations = re.split(', | and ',stations_s[0])
        
        dic = {'type': 'skip',
               'lines': line,
               'direction': bound,
               'stations': stations}
     

        
    ## run express
    elif 'run express' in a:
        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE', a)

        ##find bound
        last_line = line[-1]
        text = str(last_line) + r' (.*?) trains'
        bound = re.findall(text, a)[0]

        ##station from 
        start = re.findall(r'from (.*?) to', a)

        ##station to
        end = re.findall('to (.*)', a)
        

        dic = {'type': 'run express',
               'lines': line,
               'direction': bound,
               'from': start,
               'to': end}
    
    
    ## run local
    elif 'run local' in a:
        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE', a)

        ##find bound
        last_line = line[-1]
        text = str(last_line) + r' (.*?) trains'
        bound = re.findall(text, a)[0]

        ##station from 
        start = re.findall(r'from (.*?) to', a)

        ##station to
        end = re.findall('to (.*)', a)

        dic = {'type': 'run local',
               'lines': line,
               'direction': bound,
               'from': start,
               'to': end}

        
    ## operation frequency    
    elif 'operates every' in a:
        
        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE Service', a)
        line = [x.strip(' Service') for x in line]

        ##frequency
        freq = re.findall('every (.*) between',a)

        ##start from
        start = re.findall(r'between (.*?) and', a)

        ##end to
        end = re.findall('%s and (.*)'%start[0], a)

        dic = {'type' : 'operate frequency',
              'lines' : line,
              'frequency' : freq[0],
              'from' : start[0],
              'to' : end[0]}
        
        
    ## Service end early
    elif 'ends early' in a or 'end early' in a:

        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE Service', a)
        line = [x.strip(' Service') for x in line]

        ##No direction

        ##start from
        start = re.findall(r'between (.*?) and', a)

        ##alternative lines
        s_2 = a.replace(line[0],'')
        alt_lines = re.findall('[a-zA-Z0-9]*_LINE', s_2)
        
        ##end to
        end = re.findall('and (.*) %s'%alt_lines[0], a)

        dic = {'lines': line,
               'from': start[0],
               'to' : end,
               'alt lines' : alt_lines,
               'type' : 'end early'}    
    
    
    ## NO TRAIN
    elif 'No trains' in a:

        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE', a)


        ##station from
        start = re.findall(r'between (.*?) and', a)

        ##station to
        end = re.findall('and (.*)', a)

        dic = {'lines' : line,
               'from': start[0],
               'to': end[0],
              'type': 'No train'}
     
    ## ENTRANCE CLOSURE
    elif 'Entrance Closure' in a:

        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE', a)

        ##stop
        stop = re.findall('_LINE (.*?) Station', a)

        dic = {'lines' : line,
                'station' : stop,
                'type' : 'Entrance Closure'}
 
   
    ## restore
    elif 'restored' in a:
        
        ##find lines
        lines = re.findall('[a-zA-Z0-9]*_LINE', a)
        
        ##stations
        stations_s = re.findall('at (.*)', a)[0]
        stations = re.split(', | and ',stations_s)
        
        dic = {'lines' : lines,
               'station' : stations,
               'type' : 'restore'}

    if dic == None:
        print("no category found")
        return None
    else:
        return dic

In [103]:
for i in cat.notes:
    print(parse(i))

{'type': 'run express', 'lines': ['2_LINE', '5_LINE'], 'direction': 'Uptown', 'from': ['3 Av-149 St'], 'to': ['E 180 St']}
{'type': 'run local', 'lines': ['2_LINE'], 'direction': 'Downtown', 'from': ['96 St'], 'to': ['Times Sq-42 St']}
{'type': 'skip', 'lines': ['2_LINE'], 'direction': 'Flatbush Av-bound', 'stations': ['Nereid Av', '233 St', '225 St', '219 St']}
{'type': 'replace', 'replaced by': ['2_LINE'], 'replacing': ['5_LINE'], 'time': 'weekend', 'from': 'Dyre Av', 'to': 'E 180 St'}
{'type': 'change platform', 'lines': ['4_LINE'], 'board at': 'New Lots Av-bound', 'from': 'Junius St', 'to': 'Sutter Av-Rutland Rd'}
{'lines': ['4_LINE'], 'from': 'Utica Av', 'to': 'New Lots Av', 'type': 'No train'}
{'type': 'local stops', 'lines': ['A_LINE'], 'direction': 'both', 'stations': ['155 St', '163 St']}
{'type': 'change platform', 'lines': ['D_LINE', 'N_LINE', 'R_LINE'], 'stations': ['Prospect Av'], 'reason': 'renovation'}
{'type': 'rerout', 'rerouted by': ['E_LINE'], 'rerouting': ['F_LINE']